<a href="https://colab.research.google.com/github/alberthp/Dijkstra/blob/main/Networks_Dijkstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definition of network using JSON

In [ ]:
network_data = {
  "nodes": ["A", "B", "C", "D", "E"],
  "edges": [
    {"source": "A", "destination": "B", "cost": 2},
    {"source": "A", "destination": "C", "cost": 4},
    {"source": "B", "destination": "C", "cost": 1},
    {"source": "B", "destination": "D", "cost": 7},
    {"source": "C", "destination": "E", "cost": 3},
    {"source": "D", "destination": "E", "cost": 1}
  ],
  "start_node": "A"
}

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

def generate_full_network_interactive():
    """Generates a full network interactively with node selection, confirmation, and edge generation with costs."""
    global network_data_variable
    network_data_variable = None
    nodes = []
    edges_data = []

    num_nodes_widget = widgets.IntSlider(
        value=5,
        min=2,
        max=10,
        step=1,
        description='Number of Nodes:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    confirm_nodes_button = widgets.Button(description="Confirm Nodes")
    network_output = widgets.Output()
    process_dijkstra_button = widgets.Button(description="Process Dijkstra")
    dijkstra_output = widgets.Output()

    display(num_nodes_widget, confirm_nodes_button, network_output, process_dijkstra_button, dijkstra_output)

    def on_confirm_nodes_clicked(b):
        global nodes
        global edges_data
        num_nodes = num_nodes_widget.value
        nodes = [chr(ord('A') + i) for i in range(num_nodes)]
        edges_data = []

        with network_output:
            clear_output()
            if nodes:
                print("Generated Nodes:", nodes)
                G = nx.Graph()
                G.add_nodes_from(nodes)
                possible_edges = []
                for i in range(len(nodes)):
                    for j in range(i + 1, len(nodes)):
                        if random.random() < 0.6:  # Adjust probability for denser/sparser networks
                            cost = random.randint(1, 10)
                            edges_data.append({"source": nodes[i], "destination": nodes[j], "cost": cost})
                            G.add_edge(nodes[i], nodes[j], weight=cost)

                print("\nGenerated Edges:")
                for edge in edges_data:
                    print(f"{edge['source']} --({edge['cost']})-- {edge['destination']}")

                pos = nx.spring_layout(G)
                nx.draw(G, pos, with_labels=True, node_size=1500, node_color="lightblue", font_size=10, font_weight="bold")
                nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)})
                plt.title("Generated Network")
                plt.show()

                # Enable the Dijkstra button and ask for start node
                process_dijkstra_button.disabled = False
                global start_node_widget
                start_node_widget = widgets.Dropdown(
                    options=nodes,
                    value=nodes[0] if nodes else None,
                    description='Start Node:',
                    disabled=not nodes,
                )
                display(start_node_widget)
            else:
                print("No nodes generated.")

    def on_process_dijkstra_clicked(b):
        with dijkstra_output:
            clear_output()
            if nodes and edges_data and hasattr(globals(), 'start_node_widget') and start_node_widget.value:
                start_node = start_node_widget.value
                network_data = {
                    "nodes": nodes,
                    "edges": edges_data,
                    "start_node": start_node
                }
                global network_data_variable
                network_data_variable = network_data
                print("Ready to process Dijkstra's algorithm with the generated network:")
                print(json.dumps(network_data_variable, indent=2))
                print("\nYou can now run the Dijkstra algorithm code in the next cell using the 'network_data_variable'.")
            else:
                print("Please confirm the number of nodes first, and ensure a start node is selected.")

    confirm_nodes_button.on_click(on_confirm_nodes_clicked)
    process_dijkstra_button.on_click(on_process_dijkstra_clicked)
    process_dijkstra_button.disabled = True # Disable until nodes are confirmed

    print("Please use the slider to select the number of nodes and then click 'Confirm Nodes'.")
    return network_data_variable

# Run the interactive network generator
network_data_variable = generate_full_network_interactive()

Computation of Dijkstra

In [ ]:
import json
import heapq
import networkx as nx
import matplotlib.pyplot as plt
import time
from IPython.display import display, HTML
import pandas as pd

def dijkstra_visualized_table_side_accumulative(graph_adj, start_node, edges_list):
    distances = {node: float('inf') for node in graph_adj}
    distances[start_node] = 0
    priority_queue = [(0, start_node)]
    previous_nodes = {node: None for node in graph_adj}
    visited_nodes = set()
    step_number = 0
    table_rows = []

    G = nx.Graph()
    for node in graph_adj:
        G.add_node(node)
    for u, v, weight in edges_list:
        G.add_edge(u, v, weight=weight)
    pos = nx.spring_layout(G)
    edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}

    def draw_step(current_node=None, updated_neighbors=None, shortest_paths=None, table_df=None):
        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        nx.draw(G, pos, with_labels=True, node_size=1500, node_color=["lightgreen" if n == start_node else "lightblue" if n in visited_nodes else "white" for n in G.nodes()],
                font_size=10, font_weight="bold", edgecolors="black")
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

        if current_node:
            nx.draw_networkx_nodes(G, pos, nodelist=[current_node], node_color="yellow", node_size=1800)

        if updated_neighbors:
            nx.draw_networkx_nodes(G, pos, nodelist=updated_neighbors, node_color="orange", node_size=1200)

        if shortest_paths:
            for target, path in shortest_paths.items():
                if path:
                    path_edges = list(zip(path, path[1:]))
                    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=3)

        plt.title(f"Dijkstra's Step {step_number}")

        if table_df is not None:
            plt.subplot(1, 2, 2)
            plt.axis('off')
            plt.table(cellText=table_df.values, colLabels=table_df.columns, cellLoc='center', loc='center', colWidths=[0.1]*len(table_df.columns))

        plt.tight_layout()
        plt.pause(1)
        plt.close()

    # Initial Step
    step_number += 1
    initial_row = {"Step": step_number, "N'": ", ".join(sorted(list(visited_nodes)))}
    for node in graph_adj:
        initial_row[node] = f"∞ / None" if node != start_node else "0 / None"
    table_rows.append(initial_row)
    df = pd.DataFrame(table_rows)
    draw_step(current_node=start_node, shortest_paths={}, table_df=df)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node in visited_nodes:
            continue

        visited_nodes.add(current_node)
        shortest_paths_so_far = {k: get_shortest_path(previous_nodes, k) for k in visited_nodes if k != start_node}
        step_number += 1
        current_row = {"Step": step_number, "N'": ", ".join(sorted(list(visited_nodes)))}
        for node in graph_adj:
            current_row[node] = f"{distances[node]:.2f} / {previous_nodes[node]}" if distances[node] != float('inf') else "∞ / None"
        table_rows.append(current_row)
        df = pd.DataFrame(table_rows)
        draw_step(current_node=current_node, shortest_paths=shortest_paths_so_far, table_df=df)

        if current_distance > distances[current_node]:
            continue

        updated_neighbors_in_step = []
        for neighbor, weight in graph_adj[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))
                updated_neighbors_in_step.append(neighbor)
                step_number += 0.5 # Indicate an update step
                update_row = {"Step": f"{step_number:.1f} (Update)", "N'": ", ".join(sorted(list(visited_nodes)))}
                for node in graph_adj:
                    update_row[node] = f"{distances[node]:.2f} / {previous_nodes[node]}" if distances[node] != float('inf') else "∞ / None"
                table_rows.append(update_row)
                df = pd.DataFrame(table_rows)
                draw_step(current_node=current_node, updated_neighbors=updated_neighbors_in_step, shortest_paths=shortest_paths_so_far, table_df=df)


        if len(visited_nodes) == len(graph_adj):
            break

    plt.show()

    print("\nFinal Table:")
    df_final = pd.DataFrame(table_rows)
    display(HTML(df_final.to_html(index=False)))

    return distances, previous_nodes

def load_network_from_variable(network_data):
    graph_adj = {node: {} for node in network_data['nodes']}
    edges_list = []
    for edge in network_data['edges']:
        source = edge['source']
        destination = edge['destination']
        cost = edge['cost']
        graph_adj[source][destination] = cost
        edges_list.append((source, destination, cost))
    start_node = network_data.get('start_node')
    return graph_adj, start_node, edges_list

def get_shortest_path(previous_nodes, target):
    path = []
    current = target
    while current:
        path.insert(0, current)
        current = previous_nodes[current]
    return path

if __name__ == "__main__":
    # Define your network_data in a separate cell and run it first
    if 'network_data' in locals():
        graph_adj, start_node, edges_list = load_network_from_variable(network_data)

        if start_node and start_node in graph_adj:
            shortest_distances, previous_nodes = dijkstra_visualized_table_side_accumulative(graph_adj, start_node, edges_list)

            print(f"\nFinal Shortest distances from {start_node}:")
            for node, distance in shortest_distances.items():
                print(f"To {node}: {distance}")

            print("\nFinal Shortest paths:")
            for node in graph_adj:
                if node != start_node:
                    path = get_shortest_path(previous_nodes, node)
                    print(f"Path to {node}: {path}")

        else:
            print("Error: Start node not defined or not in the network.")
    else:
        print("Error: The 'network_data' variable was not defined in a previous cell.")